In [232]:
import numpy as np
import json
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
from tqdm import tqdm
import re
import pandas as pd
import string
import tensorflow as tf
import math

plt.style.use("fivethirtyeight")
%matplotlib inline

In [233]:
vocabulary_size = 254
embedding_size = 300
tweet_size = 140
batch_size = 100
num_sampled = 50
num_filters = 32
num_classes_for_prediction = 10

In [234]:
# Load the input data
# twitter_data_dir = "../data/authorship_corpora/influencer_tweets.json"
# json_data=open(twitter_data_dir).read()
# twitter_data = json.loads(json_data)

input_X = np.load("../data/10_user_X_Nov_11.npy")
input_y = np.load("../data/10_user_y_Nov_11.npy")
# input_matrix = [np.zeros((vocab_size, tweet_size)) for i in np.arange(num_tweets)]

In [235]:
# Placeholders for inputs
train_inputs = tf.placeholder(tf.int32, shape=[batch_size, tweet_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, num_classes_for_prediction])

# one hot encode training labels

# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder()
# one_hot_y = ohe.fit_transform(y)

def ohe(true_index, num_classes):
    vec = np.zeros(num_classes)
    vec[true_index] = 1
    return vec
X = X = np.array([arr for arr in input_X])
y = np.array([ohe(tweet, num_classes_for_prediction) for tweet in input_y])

In [236]:
# input: vocab * n chars of specific tweets
embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
embeddings.get_shape()

TensorShape([Dimension(254), Dimension(300)])

In [237]:
embed = tf.nn.embedding_lookup(embeddings, train_inputs)
embed = tf.reshape(embed, [100, 140, 300, 1])
embed.get_shape()

TensorShape([Dimension(100), Dimension(140), Dimension(300), Dimension(1)])

In [238]:
# Convolutional Layer #1
with tf.name_scope("conv-maxpool-1"):
    W = tf.Variable(tf.truncated_normal([2, embedding_size, 1, num_filters], stddev=0.05), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
    conv = tf.nn.conv2d(
        embed, 
        W, 
        strides=[1, 1, 1, 1], 
        padding="VALID", 
        name="conv1")
    h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
    pooled = tf.nn.max_pool(
        h,
        ksize=[1, 139, 1, 1],
        strides=[1, 1, 1, 1],
        padding='VALID',
        name="pool1")

pooled.get_shape()

TensorShape([Dimension(100), Dimension(1), Dimension(1), Dimension(32)])

In [239]:
# Fully connected layer 1
h_pool_flat = tf.reshape(pooled, [batch_size, num_filters])

# Multiply w * h_pool_flat and add bias terms
# 100*32 * 32*10 -> 100*10 + 10*1 = fc_1_output
with tf.name_scope("fc-1"):
    W = tf.Variable(tf.truncated_normal([num_filters, num_classes_for_prediction], stddev=0.05), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[num_classes_for_prediction]), name="b")
    fc_1_output = tf.nn.relu(tf.nn.xw_plus_b(h_pool_flat, W, b), name="fc-1-out")

fc_1_output.get_shape()

TensorShape([Dimension(100), Dimension(10)])

In [240]:
# cross entropy loss function
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
    _sentinel=None,
    labels=train_labels,
    logits=fc_1_output,
    dim=-1,
    name=None
))

In [276]:
preds = tf.argmax(fc_1_output, 1)
true_vals = tf.argmax(train_labels, 1)
acc = tf.reduce_mean(tf.cast(tf.equal(preds, true_vals), tf.float32))

In [277]:
# SGD optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.25).minimize(loss)

In [278]:
# dimension confirmation
print("y_dim", np.array(y[:100]).shape)
print("x_dim", X.shape)

y_dim (100, 10)
x_dim (89937, 140)


In [279]:
# Train data
session = tf.Session()
init = tf.global_variables_initializer()
session.run(init)

for i in tqdm(np.arange(0, len(X), batch_size)):
    feed_dict = {train_inputs: X[i:i+batch_size], train_labels: y[i:i+batch_size]}
    ack, _, curr_loss = session.run([acc, optimizer, loss], feed_dict=feed_dict)
    if (i%(batch_size * 1000)):
        print(curr_loss)
        print(ack)

  0%|          | 1/900 [00:00<02:27,  6.11it/s]

4.1155
0.12


  0%|          | 2/900 [00:00<02:32,  5.87it/s]

5.50915
0.45


  0%|          | 3/900 [00:00<02:36,  5.73it/s]

...

1.97234
0.24


 39%|███▉      | 350/900 [01:01<01:36,  5.71it/s]

2.03009
0.28


 39%|███▉      | 351/900 [01:01<01:36,  5.71it/s]

KeyboardInterrupt: 

In [260]:
curr_embeddings = embeddings.eval(session=session)

array([[ 0.82908088,  0.03529214, -0.85275292, ...,  0.1103657 ,
         0.0549611 , -0.08154214],
       [ 0.29706368, -0.65710288,  0.67986101, ...,  0.93758243,
         0.11322942, -0.78858614],
       [ 0.43298236,  0.43333718, -0.40410671, ...,  0.25863799,
        -0.32836887, -0.60043108],
       ..., 
       [ 0.76486349, -0.01643276,  0.65405154, ..., -0.89540887,
         0.54599524, -0.78034663],
       [ 0.49507594,  0.2293694 ,  0.85718751, ..., -0.59068561,
        -0.56061649,  0.01260757],
       [-0.25119564, -0.7028442 , -0.92124748, ...,  0.95713019,
         0.31512311,  0.69014138]], dtype=float32)

In [ ]:
# Visualize embeddings using TSNE
def plot_with_labels(low_dim_embs, labels, filename):
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label,
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.savefig(filename)

try:
    # pylint: disable=g-import-not-at-top
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt

    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
    plot_only = 500
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
    labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    plot_with_labels(low_dim_embs, labels, os.path.join(gettempdir(), 'tsne.png'))